### COMPARING MODELS

In [ ]:
# IMPORT EVERYTHING
#from Architectures.AE_ADJ import AE  # for some reason importing this makes it not work? (reconstructs everything around the brain somehow)
from Architectures.AE_ADJ_v2 import AE  # for some reason importing this makes it not work? (reconstructs everything around the brain somehow)

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from sklearn.cluster import SpectralClustering
from scipy.stats import f_oneway
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.cluster import KMeans

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
groups = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/D2-{group}-A.npy","br"))

Opening faster: 100%|██████████| 3/3 [00:00<00:00, 41.81it/s]


In [3]:
models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9")]

Models loaded successfully.


In [ ]:
def find_clusters(data, n_clusters=3):
    """
    Perform clustering on MRI latent space data using Spectral Clustering and calculate p-values.
    
    Parameters:
    data (numpy.ndarray): A matrix of shape (n_samples, m_features).
    n_clusters (int): Number of clusters to form.
    
    Returns:
    tuple: Cluster labels for each sample, p-values for samples.
    """
    
    # Perform Spectral Clustering
    #spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', n_neighbors=10) # random_state=42
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='rbf', gamma=1.0)

    # import warnings
    # with warnings.catch_warnings():
    #     warnings.filterwarnings("ignore", message="Graph is not fully connected*")
    #     spectral = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=42)
    #     labels = spectral.fit_predict(data)

    labels = spectral.fit_predict(data)
    
    # Calculate p-values for each feature using ANOVA
    p_values_samples = []
    for i in range(data.shape[1]):
        feature_data = [data[labels == cluster, i] for cluster in range(n_clusters)]
        f_stat, p_value = f_oneway(*feature_data)
        p_values_samples.append(p_value)
    
    return labels, np.array(p_values_samples)

def Silhouette(data, labels):
    return silhouette_score(data, labels)

def Inertia(data, labels):
    # Inertia calculated using the labels from SpectralClustering
    inertia = 0
    n_clusters = len(set(labels))
    for i in range(n_clusters):
        cluster_data = data[labels == i]
        cluster_center = cluster_data.mean(axis=0)
        inertia += ((cluster_data - cluster_center) ** 2).sum()
    return inertia

def ARI(data, labels, true_labels):
    return adjusted_rand_score(true_labels, labels)

def compare_models(models_list, groups, reconstruction_metrics, clustering_metrics, n_runs=20):
    # To run this function
    # 1. add name to each model, model.name="something"
    # 2. add .VAE_model = True to your vae models, .VAE_model = false to others
    # 3. add in the encode(), decode() functions (you can copy from ae_adj_v2.py), or you can modify compare_models to your likings
    
    metric_results = []
    group_labels = {"CN": 0, "AD": 1, "MCI": 2}
    true_labels = []
    
    for label, image in groups.items():
        true_labels.extend([group_labels[label]] * image.shape[0])

    for i, model in enumerate(models_list):
        model_result = {"Model": model.name}

        for label, image in groups.items():
            latent = model.encode(image)
            recon = model.autoencode(image)

            for recon_metric in reconstruction_metrics:
                metric_name = f"{label}_{recon_metric.__name__}"
                metric_value = recon_metric(image, recon).numpy()
                model_result[metric_name] = metric_value
                print(f"Reconstruction metric computed: {metric_name}")

        combined_data = np.concatenate(list(groups.values()), axis=0)
        combined_data_scaled =  StandardScaler().fit_transform(combined_data.reshape(-1, combined_data.shape[-1])).reshape(combined_data.shape)
        
        latent = model.encode(combined_data_scaled)
        if model.VAE_model:
            latent = latent[2]
        
        # Run this shit several times to get the average result for UMAP and clustering, gosh
        dim_red_latent_all_runs = []
        for i in range(n_runs):
            # Run UMAP with a different random state each time
            dim_red_latent = umap.UMAP(n_components=2).fit_transform(latent) #, random_state=i
            dim_red_latent_all_runs.append(dim_red_latent)

        # After collecting all the UMAP runs, now calculate the average for each metric
        for clustering_metric in clustering_metrics:
            metric_name = clustering_metric.__name__

            # Initialize lists to store metric values for each run
            metric_values = []

            for i in range(n_runs):
                cluster_labels, _ = find_clusters(dim_red_latent_all_runs[i])
                
                # Compute the clustering metric for each run
                if clustering_metric == ARI:
                    metric_value = clustering_metric(dim_red_latent_all_runs[i], cluster_labels, true_labels)
                else:
                    metric_value = clustering_metric(dim_red_latent_all_runs[i], cluster_labels)

                metric_values.append(metric_value)

            # Calculate the average metric across all runs
            average_metric_value = np.mean(metric_values)

            # Store the averaged metric in the model results
            model_result[metric_name] = average_metric_value
            
        metric_results.append(model_result)

        # dim_red_latent = umap.UMAP(n_components=2).fit_transform(latent)
        # cluster_labels, _ = find_clusters(dim_red_latent)
        # for clustering_metric in clustering_metrics:
        #     metric_name = clustering_metric.__name__
        #     if clustering_metric == ARI:
        #         metric_value = clustering_metric(dim_red_latent, cluster_labels, true_labels)
        #     else:
        #         metric_value = clustering_metric(dim_red_latent, cluster_labels)
        #     model_result[metric_name] = metric_value
        # metric_results.append(model_result)

    return pd.DataFrame(metric_results)

In [11]:
results = []
for i in range(3):
    
    result = compare_models(
        models_list, groups,
        reconstruction_metrics=[SSIM_loss], 
        clustering_metrics=[Silhouette, Inertia, ARI],
        n_runs = 20,
    )
    results.append(result)
    print(display(result))

31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Reconstruction metric computed: CN_SSIM_loss
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Reconstruction metric computed: MCI_SSIM_loss
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Reconstruction metric computed: AD_SSIM_loss
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step
[0.29784888, 0.5001807, 0.5016957, 0.25662562, 0.19980179, 0.3705386, 0.34800208, 0.27892596, 0.32436863, 0.24522805, 0.29731652, 0.40374586, 0.52475256, 0.37759748, 0.26114786, 0.26806659, 0.44718316, 0.29341194, 0.27135792, 0.35805756]
[130737.97440405376, 54859.970703125, 60298.9775390625, 116422.23746913671, 141184.1485406533, 100076.63720703125, 115981.27559633553, 134016.65539745986, 115111.83945189044, 131639.43957255408, 118155.16649168357, 91282.86279296875

,Model,CN_SSIM_loss,MCI_SSIM_loss,AD_SSIM_loss,Silhouette,Inertia,ARI
0,public_repo,0.451775,0.563819,0.562078,0.341293,108349.287676,0.308663


None
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
Reconstruction metric computed: CN_SSIM_loss
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Reconstruction metric computed: MCI_SSIM_loss
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Reconstruction metric computed: AD_SSIM_loss
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step
[0.35941347, 0.31267306, 0.29743984, 0.2793794, 0.33985612, 0.28706983, 0.3875241, 0.2964282, 0.32756078, 0.27942652, 0.3342586, 0.48353747, 0.33933878, 0.50499976, 0.47743595, 0.48060957, 0.25143382, 0.32913172, 0.3906426, 0.2526928]
[105036.40159997344, 109946.86323583126, 135307.32077700645, 134331.40387062728, 118168.87697072327, 115986.20710480306, 89080.216796875, 111167.57866434753, 127583.59981182218, 120811.32877798565, 106536.91139517725, 65274.229980

,Model,CN_SSIM_loss,MCI_SSIM_loss,AD_SSIM_loss,Silhouette,Inertia,ARI
0,public_repo,0.451775,0.563819,0.562078,0.350543,103758.91645,0.311409


None
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
Reconstruction metric computed: CN_SSIM_loss
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Reconstruction metric computed: MCI_SSIM_loss
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Reconstruction metric computed: AD_SSIM_loss
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step
[0.50781226, 0.275681, 0.28519446, 0.47263414, 0.3741333, 0.30203485, 0.31813475, 0.27579677, 0.22377707, 0.34430367, 0.28364393, 0.22787488, 0.22134377, 0.26267764, 0.23253126, 0.5136193, 0.30698538, 0.4767039, 0.5023635, 0.33304948]
[61945.1484375, 123298.56571317464, 123714.21200043708, 61897.732421875, 105224.3564453125, 120289.8640525043, 131858.55572683737, 132686.85810047016, 126736.8886048682, 106603.73442047462, 135805.03324810043, 129580.52146999538, 

,Model,CN_SSIM_loss,MCI_SSIM_loss,AD_SSIM_loss,Silhouette,Inertia,ARI
0,public_repo,0.451775,0.563819,0.562078,0.337015,108637.764629,0.300806


None


In [6]:
c = np.array([-0.0006219732, -0.018714163, -0.042449962, 0.014990389, -0.092653364, -0.06656094, 0.18586925, -0.02196336, -0.16834322, -0.033102065, 0.07216455, 0.019028865, 0.09040255, -0.025142442, -0.0655456, -0.06343138, -0.18597804, -0.19844708, 0.28852838, 0.039201133])
c.mean(), c.std()

(-0.01363842361, 0.11348698669729729)

In [ ]:
#models_list2d = [E5] #[A9,A6,A7] #[E,E2] #, B0, B1, B2]
#models_list = models_list2d

if 'models_list' in locals(): # if the variable exists
    images, latent, titles = compare_models_reconstruction(brain_scan_id=0, models_list=models_list, test=groups["CN"], loss=SSIM) # remember to update options [test, test2d]
    #print(latent)
    plot_images(images, dimensions=(20,3), fig_size=(6, 30), titles=titles, top_title="")
    
    plot_models_training_time(models_list, fig_size=(5,5))
    compare_models_loss_history(models_list, log10=True, fig_size=(5,5))

In [ ]:
# simply add .vae_mdoel = False to your models.. then it works
for m in models_list:
    plot_multiple_datasets( 
        model=m,
        plot_functions=["PCA", "TSNE", "UMAP", "ISOMAP"],
        datasets=[groups["CN"], groups["MCI"], groups["AD"]],
        labels=["H", "MCI", "AD"],
        colors=["green", "blue", "red"],
        titles=["PCA", "t-SNE", "UMAP", "Isomap"]
    )
